In [1]:
import torch
import time
import numpy as np
from model import Graph_OurConvNet
from util.helper import calculate_avg_accuracy, update_lr, print_results, load_variables
import argparse

In [4]:
# Training settings
# parser = argparse.ArgumentParser()
# parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training.')
# parser.add_argument('--seed', type=int, default=42, help='Random seed.')
# parser.add_argument('--epochs', type=int, default=10000, help='Number of epochs to train.')
# parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.')
# parser.add_argument('--l2', type=float, default=0.005, help='Weight decay (L2 loss on parameters).')
# parser.add_argument('--hidden', type=int, default=50, help='Number of hidden units.')
# parser.add_argument('--layers', type=int, default=6, help='Number of conv layers.')
# parser.add_argument('--d_conv', type=float, default=0.5, help='Dropout Conv rate (1 - keep probability).')
# parser.add_argument('--d_edge', type=float, default=0.5, help='Dropout Edge rate (1 - keep probability).')
# parser.add_argument('--d_in', type=float, default=0.0075, help='Dropin rate (1 - keep probability).')
# parser.add_argument('--dataset', default="cora", help='Name of Dataset')
# parser.add_argument('--save_path', default="model_states/state", help='Name of Dataset')

# args = parser.parse_args()
# args.cuda = True

np.random.seed(42)
print('cuda available')
dtypeFloat = torch.cuda.FloatTensor
dtypeLong = torch.cuda.LongTensor
torch.cuda.manual_seed(42)

cuda available


In [5]:
def train(net, lr, l2, batch_iters, nb_classes, early_stopping, SAVE_PATH, verbose=False, max_iters = 1000):
    ### optimization parameters
    decay_rate = 1.25

    # Optimizer
    optimizer = net.update(lr, l2) 
    t_start = time.time()
    t_start_total = time.time()
    average_loss_old = torch.tensor(1e4).cuda()
    best = running_train_acc = running_train_loss = running_val_loss = 0.0
    tab_results = []

    for iteration in range(1, max_iters):  # loop over the dataset multiple times
        # forward, loss
        net.train()
        pred_y = net.forward(features_x, E_start, E_end, E_identity, E_dropin)
        loss = net.loss(pred_y[idx_train], train_y[idx_train], None) 
        train_acc = calculate_avg_accuracy(nb_classes, train_y[idx_train], pred_y[idx_train]) # training acc
        running_train_acc += train_acc    
        running_train_loss += loss.item()

        # backward, update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # validation eval
        net.eval()
        y_eval = net.forward(features_x, E_start, E_end, E_identity, E_dropin)
        val_loss = net.loss(y_eval[idx_val], train_y[idx_val], None) 
        running_val_loss += val_loss.item()

        # learning rate, print results
        if not iteration%batch_iters:
            val_accuracy = calculate_avg_accuracy(nb_classes, train_y[idx_val], y_eval[idx_val])
            average_val_loss = running_val_loss/ batch_iters
            avg_train_acc = running_train_acc/ batch_iters

            # update learning rate 
            if val_accuracy < avg_train_acc and avg_train_acc > 0.8:
                optimizer, lr = update_lr(net, optimizer, average_val_loss, average_loss_old, 
                                          lr, decay_rate, early_stopping, verbose)

            # save intermediate results
            if val_accuracy > best:
                torch.save(net.state_dict(), SAVE_PATH)
                best = val_accuracy
            tab_results.append([iteration,average_val_loss,100* val_accuracy, time.time()-t_start_total])

            if verbose:
                print_results(iteration, batch_iters, avg_train_acc, running_train_loss, val_accuracy, lr, t_start)
            if lr < torch.tensor(early_stopping).cuda() and avg_train_acc - val_accuracy > 0.05:
                print("Early Stopping at %d. Highest Val: %.3f " % (iteration, max([tab_results[i][2] for i in range(len(tab_results))])))
                return max([tab_results[i][2] for i in range(len(tab_results))])
                break

            # reset counters
            t_start = time.time()
            running_train_acc = running_train_loss = running_val_loss = 0.0
            average_loss_old = average_val_loss
    return max([tab_results[i][2] for i in range(len(tab_results))])

In [10]:
norm = torch.sum(E_end.t(), 1).reshape(-1,1)

In [13]:
norm = torch.max(norm, torch.ones(norm.shape).cuda())

In [15]:
min(norm)

tensor([1.], device='cuda:0')

In [12]:
norm.shape

torch.Size([3327, 1])

In [9]:
features_x, train_y, E_start, E_end, E_identity, E_dropin, idx_train, idx_val, idx_test = load_variables('citeseer', 1)

net_parameters = {}
net_parameters['features'] = features_x.shape[1]
net_parameters['nb_clusters_target'] = torch.max(train_y).item()+1
net_parameters['H'] = 50
net_parameters['L'] = 4
net_parameters['Dropout_fc'] = 0.4
net_parameters['Dropout_edge'] = 0.4
net_parameters['Dropout_in'] = 0.001
lr = 0.001
l2 = 0.005
batch_iters = 20
early_stopping = 50e-6
SAVE_PATH = 'model_states/citeseer'
verbose = True
test_acc = np.zeros((10,1))


for i in range(1):
    net = Graph_OurConvNet(net_parameters, 1, 1)
    net.cuda()
        
    print("Training on", 'citeseer')
    print("Setting Layers=%d, L2=%.5f, LR=%f, Dropout_FC=%.3f, Dropout_edge=%.3f, Dropin=%.5f" % (net_parameters['L'], l2, lr, net_parameters['Dropout_fc'], net_parameters['Dropout_edge'], net_parameters['Dropout_in']))
    
    train(net, lr, l2, batch_iters, net_parameters['nb_clusters_target'],  early_stopping, SAVE_PATH, verbose, 500)
    
    # Eval
    net.load_state_dict(torch.load(SAVE_PATH))
    net.eval()
    y_eval = net.forward(features_x, E_start, E_end, E_identity, E_dropin)

    loss = net.loss(y_eval[idx_test], train_y[idx_test], None) 
    accuracy = calculate_avg_accuracy(net_parameters['nb_clusters_target'], 
                                      train_y[idx_test], y_eval[idx_test])
    print('\ntest loss = %.3f, test accuracy = %.3f' % (loss.item(), 100* accuracy))
    test_acc[i] += accuracy


/home/jerry/torch/graph-neural-networks/util/utils.py:185: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


Training on citeseer
Setting Layers=4, L2=0.00500, LR=0.001000, Dropout_FC=0.400, Dropout_edge=0.400, Dropin=0.00100

iteration= 20, train loss(20iter)= nan, lr= 0.0010000, time(20iter)= 19.38
val accuracy= 5.800
train accuracy= 16.667

iteration= 40, train loss(20iter)= nan, lr= 0.0010000, time(20iter)= 19.35
val accuracy= 5.800
train accuracy= 16.667

iteration= 60, train loss(20iter)= nan, lr= 0.0010000, time(20iter)= 19.32
val accuracy= 5.800
train accuracy= 16.667


KeyboardInterrupt: 